# Application: Interpolating Weak Reaction Rate Tables

Supernovae shine because of the radioactive decay of ${}^{56}\mathrm{Ni}$.  The electron-capture reaction:

$${}^{56}\mathrm{Ni} + e^- \rightarrow {}^{56}\mathrm{Co} + \nu_e$$

is important in determining how much ${}^{56}\mathrm{Ni}$ is created in the supernova, compared
to stable iron-group nuclei.  A similiar transformation can happen via $\beta^+$ decay:

$${}^{56}\mathrm{Ni} \rightarrow {}^{56}\mathrm{Co} + e^+ + \nu_e$$

```{note}
We call these reactions *weak nuclear reactions*, because they involve changing a neutron to a proton, and thus
the weak nuclear force.
```

These reaction rates are temperature and density dependent (in terms of electron density, $\rho Y_e$, where $Y_e$ is the electron fraction)
and evaluations of them are usually provided
as tables requiring reaction networks to interpolate.

Here we'll learn how to work with a tabulated reaction rate.

## Accessing the data

Here's a table for the above rates: {download}`56ni-56co_electroncapture.dat` from [Langanke & Martínez-Pinedo 2001](https://www.sciencedirect.com/science/article/abs/pii/S0092640X01908654).  This has been cleaned up a little
from the tables provided there, combining both the e-capture and $\beta^+$-decay into a single rate.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!head 56ni-56co_electroncapture.dat

#56ni -> 56co, e- capture
#Q=-1.624 MeV
#
#Log(rhoY)    Log(temp)     mu              dQ    Vs     Log(e-cap-rate)        Log(nu-energy-loss)  Log(gamma-energy)
#Log(g/cm^3)  Log(K)        erg             erg   erg    Log(1/s)               Log(erg/s)           Log(erg/s)
1.000000      7.000000      -4.806530e-09  0.00  0.00   -8.684000              -1.486129e+01         -100.00
1.000000      8.000000      -9.292624e-08  0.00  0.00   -9.164000              -1.533229e+01         -100.00
1.000000      8.301030      -2.146917e-07  0.00  0.00   -9.291000              -1.544729e+01         -100.00
1.000000      8.602060      -4.902661e-07  0.00  0.00   -9.387000              -1.551729e+01         -100.00
1.000000      8.845098      -8.058948e-07  0.00  0.00   -8.777000              -1.485829e+01         -100.00


We see that the data is in columns with most quantities stored as logs.  We're interest in the first column, $\log(\rho Y_e)$,
the second column, $\log(T)$, and the sixth column, $\log(\lambda)$ (the reaction rate).

```{note}
The $\log()$ are base-10 logarithms.
```

```{tip}
We'll use `np.genfromtxt()` to read this, which can interpret a comment line as labels for the data, giving us an
easy way to index the columns.
```

In [3]:
data = np.genfromtxt("56ni-56co_electroncapture.dat", skip_header=3, names=True)

From the table, we can learn that there are 13 temperatures and 11 densities in the tabulation.

In [4]:
ntemp = 13
nrho = 11

We can extract the unique densities ($\log(\rho Y_e)$) from the `LogrhoY` column, by taking every `ntemp` value:

In [5]:
logrho = data["LogrhoY"][::ntemp]
logrho

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

and the unique temperatures ($\log(T)$) from the `Logtemp` column:

In [6]:
logT = data["Logtemp"][0:ntemp]
logT

array([ 7.      ,  8.      ,  8.30103 ,  8.60206 ,  8.845098,  9.      ,
        9.176091,  9.30103 ,  9.477121,  9.69897 , 10.      , 10.477121,
       11.      ])

```{note}
The temperatures are not evenly spaced.  We need to make sure that we take that
into account when we do the interpolation.
```

Finally, we'll access the rate, $\lambda$, from the `Logecaprate` column, and reshape it into a two-dimensional table:

In [7]:
ec = data["Logecaprate"].reshape(nrho, ntemp)

Here are a few of the values:

In [8]:
ec[0:3, 0:3]

array([[-8.684, -9.164, -9.291],
       [-7.705, -8.165, -8.291],
       [-6.834, -7.171, -7.293]])

In this form, density varies in the first dimension (rows) and temperature varies in the second dimension (columns).

## Bilinear interpolation

We'll do bilinear interpolation, using taking the 4 points in the table that surround the $(\rho Y_e, T)$ at which we wish to evaluate
the rate and fit it with a function:

\begin{align*}
\lambda(\rho Y_e, T) = {}&a (\log(\rho Y_e) - \log(\rho Y_e)_i)(\log(T) - \log(T_j)) + \\
                       &b (\log(\rho Y_e) - \log(\rho Y_e)_i) + \\
                       &c (\log(T) - \log(T)_j) + \\
                       &d
\end{align*}

here we take $(i, j)$ as the reference point for the interpolant.

We need to find the 4 unknowns, $\{a, b, c, d\}$, using the information from the table.

Visually, this appears as:

![bilinear interpolation](bilinear_ecapture.png)

where the $\times$ is where we want to know the rate.

We can do this algebraically, by evaluating our interpolant at our 4 points:

\begin{align*}
\lambda_{i,j} &= d \\
\lambda_{i+1,j} &= b \Delta (\log\rho Y_e) + d \\
\lambda_{i,j+1} &= c \Delta (\log T) + d \\
\lambda_{i+1,j+1} &= a \Delta (\log\rho Y_e) \Delta (\log T) + b \Delta (\log \rho Y_e) + c \Delta (\log T) + d
\end{align*}

Then we see:

$$d = \lambda_{i,j}$$

$$b = \frac{\lambda_{i+1,j} - \lambda_{i,j}}{\Delta (\log \rho Y_e)}$$

$$c = \frac{\lambda_{i,j+1} - \lambda_{i,j}}{\Delta (\log T)}$$

$$a = \frac{\lambda_{i+1,j+1} - \lambda_{i+1,j} - \lambda_{i,j+1} + \lambda_{i,j}}{\Delta(\log \rho Y_e) \Delta( \log T)}$$

In [37]:
def interp(logrhoY_v, logT_v, lograte, logrhoY0, logT0):
    """interpolate lograte to find the rate at a point (logrhoY0, logT0).
    Here logrhoY_v and logT_v are the points where the rate is tabulated"""

    i = np.argwhere(logrhoY_v > logrhoY0)[0][0] - 1
    j = np.argwhere(logT_v > logT0)[0][0] - 1

    dlogrhoY = logrhoY_v[i+1] - logrhoY_v[i]
    dlogT = logT_v[j+1] - logT_v[j]

    a = (lograte[i+1, j+1] - lograte[i+1, j] - lograte[i, j+1] + lograte[i, j]) / (dlogrhoY * dlogT)
    b = (lograte[i+1, j] - lograte[i, j]) / dlogrhoY
    c = (lograte[i, j+1] - lograte[i, j]) / dlogT
    d = lograte[i, j]

    return (a * (logrhoY0 - logrhoY_v[i]) * (logT0 - logT_v[j]) +
            b * (logrhoY0 - logrhoY_v[i]) +
            c * (logT0 - logT_v[j]) +
            d)

In [38]:
interp(logrho, logT, ec, 3.5, 9.4)

2 7
1.0 0.17609099999999955
3.0 4.0
-0.14806548886655366 0.03097500000000064 6.174472289895581 -5.40263


-4.7833819981855985

In [35]:
ec[2:4, 7:9]

array([[-5.40263 , -4.315361],
       [-5.371655, -4.310459]])

In [22]:
logT

array([ 7.      ,  8.      ,  8.30103 ,  8.60206 ,  8.845098,  9.      ,
        9.176091,  9.30103 ,  9.477121,  9.69897 , 10.      , 10.477121,
       11.      ])

## Using SciPy

In [23]:
from scipy import interpolate

In [39]:
interpolate.interpn((logrho, logT), ec, (3.5, 9.4))

array([-4.783382])